In [ ]:
import os
import torch
import polars as pl
import numpy as np
import seaborn as sns
import warnings
from feature.scalers import ranged_scaler
from datetime import datetime, timedelta
from rca.causal_diagnosis import root_cause_diagnosis
warnings.filterwarnings("ignore", category=UserWarning) 

In [ ]:
cats_df = pl.read_csv("data/data.csv", separator=",")  

In [ ]:
metadata = pl.read_csv('data/metadata.csv',separator=',')


In [ ]:
cats_df.head()

In [ ]:
metadata.head()


In [ ]:
potential_causes = metadata['root_cause'].unique().to_list()

In [ ]:
for col in cats_df.columns:
    unique_vals = cats_df[col].n_unique()
    data_type = cats_df[col].dtype
    bad_dtypes = [pl.Date,pl.Datetime,pl.Utf8]
    if ((unique_vals >= 50) & (data_type not in bad_dtypes) ):
        cats_df = cats_df.with_columns(ranged_scaler(cats_df[col]))
    else:
        continue

In [ ]:
cats_df = cats_df.with_columns(
    pl.col('timestamp').str.to_datetime("%Y-%m-%d %H:%M:%S")
)

In [ ]:
cats_df.head()


In [ ]:
cats_rows_list = metadata.rows(named=True)

In [ ]:
new_metadata = []
iteration = 0
for i, row in enumerate(cats_rows_list):
    potential_causes = metadata['root_cause'].unique().to_list()
    if i == 0:
        start_time = datetime.strptime(row['start_time'],"%Y-%m-%d %H:%M:%S")
        end_time = datetime.strptime(row['end_time'],"%Y-%m-%d %H:%M:%S")
        delta = end_time - start_time
        start_time = start_time - delta
    else: 
        start_time = end_time + timedelta(seconds=1)
        end_time = datetime.strptime(row['end_time'],"%Y-%m-%d %H:%M:%S")
    anomaly = eval(row['affected'])[0]
    root_cause = row['root_cause']
    potential_causes.append(anomaly)
    mod_df = cats_df.filter( (pl.col('timestamp')>= start_time) & (pl.col('timestamp') <= end_time)).select(potential_causes)
    rca_mod = root_cause_diagnosis(input_df = mod_df,cols_to_exclude=[])
    rca_mod.fit()

    root_rank_df = rca_mod.root_rank_score
    root_rank_df=root_rank_df[root_rank_df['Column']!=anomaly][0:3].reset_index()
    potential_cause1 = root_rank_df['Column'][0]
    potential_cause2 = root_rank_df['Column'][1]
    potential_cause3 = root_rank_df['Column'][2]
    if root_cause == potential_cause1:
        row['cause_1'] = 1
    if root_cause == potential_cause2:
        row['cause_2'] = 1
    if root_cause == potential_cause3:
        row['cause_3'] = 1
    new_metadata.append(row)
    if iteration%5 == 0:
        print(iteration)
    iteration+=1